# DS6600 FINAL PROJECT

## BY HIEU VU

In [ ]:
import os
import json
import time
from datetime import datetime, timedelta, timezone

import pandas as pd
import requests
from dateutil import parser
from sqlalchemy import create_engine, text
import numpy as np

import dash
from dash import html, dcc
import plotly.express as px
import plotly.graph_objects as go

engine = create_engine("sqlite:///sports_pipeline.db")




### Configuration setup

In [2]:
BALLDONTLIE_API_KEY = os.getenv("SPORTS_KEY")
ODDS_API_KEY = os.getenv("BETTING_ODDS_KEY")

BALLDONTLIE_BASE = "https://api.balldontlie.io/v1"
ODDS_BASE = "https://api.the-odds-api.com/v4"

DB_URL = os.getenv("DB_URL", "sqlite:///sports_pipeline.db")

ODDS_SPORT_KEY = "basketball_nba" 


### Helper Functions

In [ ]:
def req_json(url, headers=None, params=None, timeout=30):
    r = requests.get(url, headers=headers, params=params, timeout=timeout)

    if not (200 <= r.status_code < 300):
        preview = r.text[:500]
        raise RuntimeError(
            f"HTTP {r.status_code} for {r.url}\n"
            f"Content-Type: {r.headers.get('Content-Type')}\n"
            f"Body (first 500 chars):\n{preview}"
        )

    ctype = (r.headers.get("Content-Type") or "").lower()
    if "json" not in ctype:
        preview = r.text[:500]
        raise RuntimeError(
            f"Non-JSON response for {r.url}\n"
            f"Content-Type: {r.headers.get('Content-Type')}\n"
            f"Body (first 500 chars):\n{preview}"
        )

    return r.json()

def to_utc_dt(dt_str):
    dt = parser.isoparse(dt_str)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def normalize_team_name(name: str) -> str:
    n = name.strip().lower()
    n = n.replace(".", "").replace("-", " ")
    n = " ".join(n.split())
    return n

def american_to_implied_prob(odds: float) -> float:
    if odds is None:
        return None
    odds = float(odds)
    if odds > 0:
        return 100.0 / (odds + 100.0)
    else:
        return (-odds) / ((-odds) + 100.0)

def remove_vig_two_way(p_a: float, p_b: float):
    if p_a is None or p_b is None:
        return None, None
    s = p_a + p_b
    if s <= 0:
        return None, None
    return p_a / s, p_b / s


### Extraction Functions

In [ ]:
def fetch_balldontlie_teams():
    if not BALLDONTLIE_API_KEY:
        raise RuntimeError("Missing BALLDONTLIE_API_KEY")

    headers = {"Authorization": BALLDONTLIE_API_KEY}
    teams = []
    cursor = 0

    while True:
        data = req_json(
            f"{BALLDONTLIE_BASE}/teams",
            headers=headers,
            params={"per_page": 100, "cursor": cursor}
        )
        teams.extend(data.get("data", []))

        next_cursor = (data.get("meta") or {}).get("next_cursor")
        if not next_cursor:
            break
        cursor = next_cursor

    df = pd.json_normalize(teams)
    df["team_name_norm"] = df["full_name"].astype(str).str.lower().str.replace(r"[\.\-]", " ", regex=True).str.split().str.join(" ")
    return df

def fetch_balldontlie_games(start_date_utc: datetime, end_date_utc: datetime):
    if not BALLDONTLIE_API_KEY:
        raise RuntimeError("Missing BALLDONTLIE_API_KEY")

    headers = {"Authorization": BALLDONTLIE_API_KEY}
    games = []
    cursor = 0

    date_from = start_date_utc.date().isoformat()
    date_to = end_date_utc.date().isoformat()

    while True:
        params = {
            "per_page": 100,
            "cursor": cursor,
            "start_date": date_from,
            "end_date": date_to,
        }
        data = req_json(f"{BALLDONTLIE_BASE}/games", headers=headers, params=params)
        games.extend(data.get("data", []))

        next_cursor = (data.get("meta") or {}).get("next_cursor")
        if not next_cursor:
            break
        cursor = next_cursor
        time.sleep(0.2)

    df = pd.json_normalize(games)

    wanted = [
        "id", "date", "season", "postseason", "status",
        "home_team.id", "home_team.full_name", "home_team.abbreviation",
        "visitor_team.id", "visitor_team.full_name", "visitor_team.abbreviation",
        "home_team_score", "visitor_team_score",
    ]
    cols = [c for c in wanted if c in df.columns]
    df = df[cols].copy()

    if "date" in df.columns:
        df["game_start_utc"] = df["date"].map(lambda x: to_utc_dt(x).isoformat())
    df["home_team_name_norm"] = df.get("home_team.full_name", "").map(normalize_team_name)
    df["away_team_name_norm"] = df.get("visitor_team.full_name", "").map(normalize_team_name)

    def winner(row):
        try:
            hs, as_ = float(row["home_team_score"]), float(row["visitor_team_score"])
            if pd.isna(hs) or pd.isna(as_):
                return None
            return "home" if hs > as_ else ("away" if as_ > hs else "tie")
        except Exception:
            return None

    if "home_team_score" in df.columns and "visitor_team_score" in df.columns:
        df["winner"] = df.apply(winner, axis=1)

    return df

def fetch_odds_events(regions="us", markets="h2h", odds_format="american", date_format="iso"):
    params = {
        "apiKey": ODDS_API_KEY,
        "regions": regions,
        "markets": markets,
        "oddsFormat": odds_format,
        "dateFormat": date_format,
    }
    data = req_json(f"{ODDS_BASE}/sports/{ODDS_SPORT_KEY}/odds", params=params)
    return data  

def odds_events_to_tables(events_json):
    rows_events = []
    rows_books = []
    rows_quotes = []

    for ev in events_json:
        event_id = ev.get("id")
        commence = ev.get("commence_time")
        home = ev.get("home_team")
        away = None
        # some sports list teams array; NBA typically has away team as the other one
        teams = ev.get("teams", [])
        if teams and home in teams and len(teams) == 2:
            away = teams[0] if teams[1] == home else teams[1]

        rows_events.append({
            "event_id": event_id,
            "commence_time_utc": to_utc_dt(commence).isoformat() if commence else None,
            "home_team": home,
            "away_team": away,
            "home_team_name_norm": normalize_team_name(home) if home else None,
            "away_team_name_norm": normalize_team_name(away) if away else None,
            "sport_key": ev.get("sport_key"),
            "sport_title": ev.get("sport_title"),
        })

        for bk in ev.get("bookmakers", []):
            bookmaker_key = bk.get("key")
            bookmaker_title = bk.get("title")
            last_update = bk.get("last_update")

            rows_books.append({
                "event_id": event_id,
                "bookmaker_key": bookmaker_key,
                "bookmaker_title": bookmaker_title,
                "last_update_utc": to_utc_dt(last_update).isoformat() if last_update else None,
            })

            for mkt in bk.get("markets", []):
                mkt_key = mkt.get("key")
                for out in mkt.get("outcomes", []):
                    rows_quotes.append({
                        "event_id": event_id,
                        "bookmaker_key": bookmaker_key,
                        "market_key": mkt_key,
                        "outcome_name": out.get("name"),
                        "outcome_price": out.get("price"),
                        "outcome_point": out.get("point"),  # for spreads/totals
                    })

    df_events = pd.DataFrame(rows_events).drop_duplicates(subset=["event_id"])
    df_books = pd.DataFrame(rows_books).drop_duplicates(subset=["event_id", "bookmaker_key"])
    df_quotes = pd.DataFrame(rows_quotes)

    # Implied probabilities for h2h
    if not df_quotes.empty:
        df_quotes["implied_prob"] = df_quotes["outcome_price"].apply(
            lambda x: american_to_implied_prob(x) if pd.notna(x) else None
        )

    return df_events, df_books, df_quotes


### Transform + Join Results

In [ ]:

def join_odds_to_games(df_games, df_odds_events, max_time_diff_hours=18):
    """
    Join by normalized team names + commence_time ~ game_start time.
    This produces a link table event_id -> game_id.
    """
    if df_games.empty or df_odds_events.empty:
        return pd.DataFrame(columns=["event_id", "game_id", "match_score_reason"])

    g = df_games.copy()
    o = df_odds_events.copy()

    g["game_start_dt"] = pd.to_datetime(g["game_start_utc"], utc=True, errors="coerce")
    o["commence_dt"] = pd.to_datetime(o["commence_time_utc"], utc=True, errors="coerce")

    merged = o.merge(
        g,
        left_on=["home_team_name_norm", "away_team_name_norm"],
        right_on=["home_team_name_norm", "away_team_name_norm"],
        how="left",
        suffixes=("_odds", "_game")
    )

    merged["time_diff_hours"] = (merged["commence_dt"] - merged["game_start_dt"]).abs() / pd.Timedelta(hours=1)
    merged = merged[merged["time_diff_hours"] <= max_time_diff_hours].copy()

    # If multiple matches, pick closest in time
    merged.sort_values(["event_id", "time_diff_hours"], inplace=True)
    best = merged.drop_duplicates(subset=["event_id"], keep="first")

    out = best[["event_id", "id", "time_diff_hours"]].rename(columns={"id": "game_id"})
    out["match_score_reason"] = out["time_diff_hours"].apply(lambda h: f"team+time<= {max_time_diff_hours}h (diff={h:.2f}h)")
    return out

def to_iso_z(dt: datetime) -> str:
    return dt.astimezone(timezone.utc).replace(microsecond=0).isoformat().replace("+00:00", "Z")

def fetch_odds_historical_snapshot(snapshot_utc: datetime, regions="us", markets="h2h,spreads,totals"):
    if not ODDS_API_KEY:
        raise RuntimeError("Missing ODDS_API_KEY env var")

    params = {
        "apiKey": ODDS_API_KEY,
        "regions": regions,
        "markets": markets,
        "oddsFormat": "american",
        "dateFormat": "iso",
        "date": to_iso_z(snapshot_utc),
    }

    data = req_json(f"{ODDS_BASE}/historical/sports/{ODDS_SPORT_KEY}/odds", params=params)

    events = data["data"] if isinstance(data, dict) and "data" in data else data
    return events


### Load into Database

In [6]:
DDL = """
CREATE TABLE IF NOT EXISTS teams (
  team_id INTEGER PRIMARY KEY,
  full_name TEXT,
  abbreviation TEXT,
  city TEXT,
  conference TEXT,
  division TEXT,
  name TEXT,
  team_name_norm TEXT
);

CREATE TABLE IF NOT EXISTS games (
  game_id INTEGER PRIMARY KEY,
  game_start_utc TEXT,
  season INTEGER,
  postseason BOOLEAN,
  status TEXT,
  home_team_id INTEGER,
  away_team_id INTEGER,
  home_team_name TEXT,
  away_team_name TEXT,
  home_team_abbrev TEXT,
  away_team_abbrev TEXT,
  home_score REAL,
  away_score REAL,
  winner TEXT
);

CREATE TABLE IF NOT EXISTS odds_events (
  event_id TEXT PRIMARY KEY,
  sport_key TEXT,
  sport_title TEXT,
  commence_time_utc TEXT,
  home_team TEXT,
  away_team TEXT,
  home_team_name_norm TEXT,
  away_team_name_norm TEXT
);

CREATE TABLE IF NOT EXISTS odds_bookmakers (
  event_id TEXT,
  bookmaker_key TEXT,
  bookmaker_title TEXT,
  last_update_utc TEXT,
  PRIMARY KEY (event_id, bookmaker_key)
);

CREATE TABLE IF NOT EXISTS odds_quotes (
  event_id TEXT,
  bookmaker_key TEXT,
  market_key TEXT,
  outcome_name TEXT,
  outcome_price REAL,
  outcome_point REAL,
  implied_prob REAL
);

CREATE TABLE IF NOT EXISTS event_game_link (
  event_id TEXT PRIMARY KEY,
  game_id INTEGER,
  match_score_reason TEXT
);
"""

def init_db(engine):
    with engine.begin() as conn:
        for stmt in DDL.strip().split(";"):
            s = stmt.strip()
            if s:
                conn.execute(text(s))

def load_df(engine, df, table, if_exists="append"):
    df.to_sql(table, engine, if_exists=if_exists, index=False)



In [ ]:
def fetch_games_by_season_windows(start_year=2020, end_year=2024):
    # NBA season window: Sep 1 (start year) through Jul 31 (next year) to include playoffs.
    parts = []
    for season in range(start_year, end_year + 1):
        s = datetime(season, 9, 1, tzinfo=timezone.utc)
        e = datetime(season + 1, 7, 31, tzinfo=timezone.utc)
        print(f"Fetching games window: {season}-{season+1} ({s.date()} to {e.date()})")
        df_part = fetch_balldontlie_games(s, e)
        parts.append(df_part)
        time.sleep(1.0)  # small pause between seasons
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()



In [ ]:
from sqlalchemy import text
import pandas as pd
import time
from datetime import datetime, timedelta, timezone

def insert_or_ignore(engine, df, table, key_cols):
    """
    Inserts rows using SQLite INSERT OR IGNORE to avoid UNIQUE constraint crashes.
    key_cols is just for readability; SQLite uses the table's PRIMARY KEY/UNIQUE constraints.
    """
    if df is None or df.empty:
        return 0

    cols = list(df.columns)
    placeholders = ", ".join([f":{c}" for c in cols])
    col_list = ", ".join(cols)

    sql = text(f"INSERT OR IGNORE INTO {table} ({col_list}) VALUES ({placeholders})")

    rows = df.to_dict(orient="records")
    with engine.begin() as conn:
        conn.execute(sql, rows)

    return len(df)

def load_quotes_append(engine, df_quotes):
    """
    Odds quotes table has no PK in your schema, so it can balloon with duplicates.
    We'll dedupe within this batch before append.
    """
    if df_quotes is None or df_quotes.empty:
        return 0

    df_quotes = df_quotes.drop_duplicates(
        subset=["event_id", "bookmaker_key", "market_key", "outcome_name", "outcome_point", "outcome_price"],
        keep="last"
    )

    df_quotes.to_sql("odds_quotes", engine, if_exists="append", index=False)
    return len(df_quotes)

def main():
    engine = create_engine(DB_URL)
    init_db(engine)

    with engine.begin() as conn:
        conn.execute(text("DELETE FROM odds_quotes"))
        conn.execute(text("DELETE FROM odds_bookmakers"))
        conn.execute(text("DELETE FROM odds_events"))
        conn.execute(text("DELETE FROM event_game_link"))
        conn.execute(text("DELETE FROM games"))


    df_teams = fetch_balldontlie_teams()
    load_df(engine, df_teams.rename(columns={"id": "team_id"}), "teams", if_exists="replace")

    df_games = fetch_games_by_season_windows(start_year=2020, end_year=2024)

    rename_map = {
        "id": "game_id",
        "home_team.id": "home_team_id",
        "visitor_team.id": "away_team_id",
        "home_team.full_name": "home_team_name",
        "visitor_team.full_name": "away_team_name",
        "home_team.abbreviation": "home_team_abbrev",
        "visitor_team.abbreviation": "away_team_abbrev",
        "home_team_score": "home_score",
        "visitor_team_score": "away_score",
    }

    df_games_db = df_games.rename(columns=rename_map)
    keep_games = [
        "game_id","game_start_utc","season","postseason","status",
        "home_team_id","away_team_id",
        "home_team_name","away_team_name",
        "home_team_abbrev","away_team_abbrev",
        "home_score","away_score","winner",
    ]
    df_games_db = df_games_db[keep_games].copy()
    load_df(engine, df_games_db, "games", if_exists="append")

    if not ODDS_API_KEY:
        raise RuntimeError("Missing ODDS_API_KEY")

    odds_start = datetime(2020, 6, 6, tzinfo=timezone.utc)
    odds_end   = datetime(2024, 12, 31, tzinfo=timezone.utc)

    d = odds_start

    total_events_seen = 0
    total_events_inserted = 0
    total_books_inserted = 0
    total_quotes_inserted = 0
    total_links_inserted = 0
    total_days_skipped = 0

    while d <= odds_end:
        snap = d.replace(hour=12, minute=0, second=0, microsecond=0)
        print(f"Historical odds snapshot: {snap.date()}")

        try:
            odds_json = fetch_odds_historical_snapshot(
                snap,
                regions="us",
                markets="h2h"
            )
        except Exception as e:
            print(f"[skip] {snap.date()} failed: {e}")
            total_days_skipped += 1
            d += timedelta(days=1)
            time.sleep(1.0)
            continue

        df_odds_events, df_odds_books, df_odds_quotes = odds_events_to_tables(odds_json)

        total_events_seen += len(df_odds_events)

        if df_odds_events is not None and not df_odds_events.empty:
            total_events_inserted += insert_or_ignore(engine, df_odds_events, "odds_events", ["event_id"])

        if df_odds_books is not None and not df_odds_books.empty:
            df_odds_books = df_odds_books.drop_duplicates(subset=["event_id", "bookmaker_key"], keep="last")
            total_books_inserted += insert_or_ignore(engine, df_odds_books, "odds_bookmakers", ["event_id","bookmaker_key"])

        if df_odds_quotes is not None and not df_odds_quotes.empty:
            total_quotes_inserted += load_quotes_append(engine, df_odds_quotes)

        link = join_odds_to_games(df_games, df_odds_events, max_time_diff_hours=24)
        if link is not None and not link.empty:
            total_links_inserted += insert_or_ignore(engine, link, "event_game_link", ["event_id"])

        time.sleep(1.0)
        d += timedelta(days=1)

    print("\nDONE.")
    print(f"Teams: {len(df_teams)}")
    print(f"Games: {len(df_games)}")
    print(f"Odds events seen (sum): {total_events_seen}")
    print(f"Odds events inserted: {total_events_inserted}")
    print(f"Bookmakers inserted: {total_books_inserted}")
    print(f"Quotes inserted: {total_quotes_inserted}")
    print(f"Links inserted: {total_links_inserted}")
    print(f"Days skipped (errors): {total_days_skipped}")


if __name__ == "__main__":
    main()


Fetching games window: 2020-2021 (2020-09-01 to 2021-07-31)
Fetching games window: 2021-2022 (2021-09-01 to 2022-07-31)
Fetching games window: 2022-2023 (2022-09-01 to 2023-07-31)
Fetching games window: 2023-2024 (2023-09-01 to 2024-07-31)
Fetching games window: 2024-2025 (2024-09-01 to 2025-07-31)
Historical odds snapshot: 2020-06-06
Historical odds snapshot: 2020-06-07
Historical odds snapshot: 2020-06-08
Historical odds snapshot: 2020-06-09
Historical odds snapshot: 2020-06-10
Historical odds snapshot: 2020-06-11
Historical odds snapshot: 2020-06-12
Historical odds snapshot: 2020-06-13
Historical odds snapshot: 2020-06-14
Historical odds snapshot: 2020-06-15
Historical odds snapshot: 2020-06-16
Historical odds snapshot: 2020-06-17
Historical odds snapshot: 2020-06-18
Historical odds snapshot: 2020-06-19
Historical odds snapshot: 2020-06-20
Historical odds snapshot: 2020-06-21
Historical odds snapshot: 2020-06-22
Historical odds snapshot: 2020-06-23
Historical odds snapshot: 2020-06-

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///sports_pipeline.db")

def normalize_team_name(name: str) -> str:
    if name is None:
        return None
    n = str(name).strip().lower()
    n = n.replace(".", "").replace("-", " ")
    n = " ".join(n.split())
    return n

df_games = pd.read_sql_query("""
SELECT game_id, season, game_start_utc, home_team_name, away_team_name
FROM games
""", engine)

df_games["home_team_name_norm"] = df_games["home_team_name"].map(normalize_team_name)
df_games["away_team_name_norm"] = df_games["away_team_name"].map(normalize_team_name)
df_games["game_start_dt"] = pd.to_datetime(df_games["game_start_utc"], utc=True, errors="coerce")

df_events = pd.read_sql_query("""
SELECT event_id, commence_time_utc, home_team_name_norm, away_team_name_norm
FROM odds_events
WHERE commence_time_utc IS NOT NULL
  AND home_team_name_norm IS NOT NULL
  AND away_team_name_norm IS NOT NULL
""", engine)

df_events["commence_dt"] = pd.to_datetime(df_events["commence_time_utc"], utc=True, errors="coerce")

m1 = df_events.merge(
    df_games,
    on=["home_team_name_norm", "away_team_name_norm"],
    how="inner",
)

m2 = df_events.merge(
    df_games,
    left_on=["home_team_name_norm", "away_team_name_norm"],
    right_on=["away_team_name_norm", "home_team_name_norm"],
    how="inner",
)

merged = pd.concat([m1, m2], ignore_index=True)

merged["time_diff_hours"] = (merged["commence_dt"] - merged["game_start_dt"]).abs() / pd.Timedelta(hours=1)
merged = merged[merged["time_diff_hours"] <= 24].copy()  # tweak 24→36 if needed

merged.sort_values(["event_id", "time_diff_hours"], inplace=True)
links = merged.drop_duplicates(subset=["event_id"], keep="first")[["event_id", "game_id"]].copy()

with engine.begin() as conn:
    conn.execute(text("DELETE FROM event_game_link"))

links.to_sql("_tmp_links", engine, if_exists="replace", index=False)

with engine.begin() as conn:
    conn.execute(text("""
    INSERT OR IGNORE INTO event_game_link(event_id, game_id)
    SELECT event_id, game_id FROM _tmp_links
    """))

print("event_game_link rows (time-matched):",
      pd.read_sql_query("SELECT COUNT(*) AS n FROM event_game_link", engine)["n"].iloc[0])


event_game_link rows (time-matched): 1750


In [ ]:

engine = create_engine("sqlite:///sports_pipeline.db")

df = pd.read_sql_query("""
SELECT
  l.event_id,
  l.game_id,

  g.season,
  g.game_start_utc,
  g.home_team_name,
  g.away_team_name,
  g.home_score,
  g.away_score,
  g.winner,

  ob.bookmaker_title,
  oq.outcome_name,
  oq.outcome_price,
  oq.implied_prob

FROM event_game_link l
JOIN games g
  ON g.game_id = l.game_id
JOIN odds_quotes oq
  ON oq.event_id = l.event_id
JOIN odds_bookmakers ob
  ON ob.event_id = oq.event_id
 AND ob.bookmaker_key = oq.bookmaker_key

WHERE oq.market_key = 'h2h'
""", engine)

df = df[df["winner"].isin(["home", "away"])].copy()

df["winner_team"] = np.where(df["winner"] == "home", df["home_team_name"], df["away_team_name"])

df = df.sort_values(["event_id", "bookmaker_title", "implied_prob"], ascending=[True, True, False])
favorites = df.groupby(["event_id", "bookmaker_title"], as_index=False).first()

favorites["favorite_won"] = (favorites["outcome_name"] == favorites["winner_team"]).astype(int)
favorites["season"] = pd.to_numeric(favorites["season"], errors="coerce")
favorites["implied_prob"] = pd.to_numeric(favorites["implied_prob"], errors="coerce")
favorites["outcome_price"] = pd.to_numeric(favorites["outcome_price"], errors="coerce")

favorites = favorites.dropna(subset=["implied_prob", "season"]).copy()
favorites.head()


,event_id,bookmaker_title,game_id,season,game_start_utc,home_team_name,away_team_name,home_score,away_score,winner,outcome_name,outcome_price,implied_prob,winner_team,favorite_won
0,000eb37e99f93fccfb45da8455732b88,Barstool Sportsbook,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1250.0,0.925926,Minnesota Timberwolves,1
1,000eb37e99f93fccfb45da8455732b88,BetMGM,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1000.0,0.909091,Minnesota Timberwolves,1
2,000eb37e99f93fccfb45da8455732b88,BetRivers,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1250.0,0.925926,Minnesota Timberwolves,1
3,000eb37e99f93fccfb45da8455732b88,Betfair,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-769.0,0.884925,Minnesota Timberwolves,1
4,000eb37e99f93fccfb45da8455732b88,DraftKings,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-909.0,0.900892,Minnesota Timberwolves,1


In [ ]:
#I am keeping only completed games
df = df[df["winner"].isin(["home", "away"])].copy()

df["winner_team"] = np.where(
    df["winner"] == "home",
    df["home_team_name"],
    df["away_team_name"]
)

df["outcome_price"] = pd.to_numeric(df["outcome_price"], errors="coerce")

df = df.sort_values(
    ["event_id", "bookmaker_title", "implied_prob"],
    ascending=[True, True, False]
)

favorites = df.groupby(
    ["event_id", "bookmaker_title"],
    as_index=False
).first()

favorites["favorite_won"] = (
    favorites["outcome_name"] == favorites["winner_team"]
).astype(int)

favorites.head()


,event_id,bookmaker_title,game_id,season,game_start_utc,home_team_name,away_team_name,home_score,away_score,winner,outcome_name,outcome_price,implied_prob,winner_team,favorite_won
0,000eb37e99f93fccfb45da8455732b88,Barstool Sportsbook,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1250.0,0.925926,Minnesota Timberwolves,1
1,000eb37e99f93fccfb45da8455732b88,BetMGM,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1000.0,0.909091,Minnesota Timberwolves,1
2,000eb37e99f93fccfb45da8455732b88,BetRivers,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-1250.0,0.925926,Minnesota Timberwolves,1
3,000eb37e99f93fccfb45da8455732b88,Betfair,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-769.0,0.884925,Minnesota Timberwolves,1
4,000eb37e99f93fccfb45da8455732b88,DraftKings,474379,2021,2022-03-07T00:00:00+00:00,Minnesota Timberwolves,Portland Trail Blazers,124.0,81.0,home,Minnesota Timberwolves,-909.0,0.900892,Minnesota Timberwolves,1


In [ ]:
favorites = favorites.dropna(subset=["implied_prob"]).copy()

bins = np.linspace(0, 1, 21)  # 20 bins
favorites["prob_bin"] = pd.cut(favorites["implied_prob"], bins=bins, include_lowest=True)

calib = (
    favorites.groupby("prob_bin", as_index=False)
    .agg(
        implied_prob_mean=("implied_prob", "mean"),
        actual_win_rate=("favorite_won", "mean"),
        n=("favorite_won", "size")
    )
)

calib = calib[calib["n"] >= 10].copy()

fig_calib = px.scatter(
    calib,
    x="implied_prob_mean",
    y="actual_win_rate",
    size="n",
    hover_data=["n"],
    title="Calibration: Implied Probability vs Actual Win Rate"
)

fig_calib.add_shape(
    type="line",
    x0=0, y0=0,
    x1=1, y1=1
)

fig_calib.update_layout(
    xaxis_title="Mean Implied Probability (Favorite)",
    yaxis_title="Actual Win Rate (Favorite)",
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1])
)


C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\2340263847.py:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
favorites["prob_bin"] = pd.cut(
    favorites["implied_prob"],
    bins=np.linspace(0.5, 1.0, 11)
)

bias = (
    favorites
    .groupby("prob_bin")
    .agg(
        mean_implied=("implied_prob", "mean"),
        actual_win_rate=("favorite_won", "mean"),
        n_games=("favorite_won", "size")
    )
    .reset_index()
)

bias["bias"] = bias["actual_win_rate"] - bias["mean_implied"]


C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\541347899.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [47]:
fig_bias = px.bar(
    bias,
    x="mean_implied",
    y="bias",
    text="n_games",
    labels={
        "mean_implied": "Mean Implied Probability (Favorite)",
        "bias": "Actual − Implied Win Rate"
    },
    title="Favorite Bias by Implied Probability"
)

fig_bias.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",
    yaxis_tickformat=".0%"
)


In [48]:
fig_dist = px.histogram(
    favorites,
    x="implied_prob",
    color="favorite_won",
    nbins=20,
    labels={
        "implied_prob": "Implied Probability",
        "favorite_won": "Favorite Won"
    },
    title="Distribution of Implied Probabilities by Outcome"
)

fig_dist.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white"
)


In [ ]:
def profit_per_1_unit_bet(american_odds, won: int):
    """
    Returns profit (not including stake) for a 1 unit bet.
    If won:
      +odds: profit = odds/100
      -odds: profit = 100/abs(odds)
    If lost: profit = -1 (lose your stake)
    """
    if pd.isna(american_odds):
        return np.nan
    o = float(american_odds)
    if won == 1:
        if o > 0:
            return o / 100.0
        else:
            return 100.0 / abs(o)
    else:
        return -1.0


favorites["profit_1u"] = favorites.apply(
    lambda r: profit_per_1_unit_bet(r["outcome_price"], r["favorite_won"]),
    axis=1
)

overall_roi = favorites["profit_1u"].mean()
print("Overall ROI (betting favorites, 1u each):", round(overall_roi, 4))


Overall ROI (betting favorites, 1u each): -0.0464


In [50]:
roi_by_bin = (
    favorites.dropna(subset=["profit_1u", "implied_prob"])
    .assign(prob_bin=pd.cut(favorites["implied_prob"], bins=np.linspace(0.5, 1.0, 11)))
    .groupby("prob_bin", as_index=False)
    .agg(
        mean_implied=("implied_prob", "mean"),
        mean_roi=("profit_1u", "mean"),
        n=("profit_1u", "size")
    )
)

fig_roi = px.bar(
    roi_by_bin,
    x="mean_implied",
    y="mean_roi",
    text="n",
    title="ROI by Favorite Strength (1 unit on favorites)",
    labels={"mean_implied": "Mean Implied Probability", "mean_roi": "Mean ROI (units)"}
)

fig_roi.update_layout(paper_bgcolor="white", plot_bgcolor="white")


C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\955166148.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# DASH APP

In [ ]:
from dash import Dash, html, dcc, Input, Output

def profit_per_1_unit_bet(american_odds, won: int):
    """
    Profit (excluding stake) on a 1-unit bet.
    Win:
      +odds => odds/100
      -odds => 100/abs(odds)
    Loss: -1
    """
    if pd.isna(american_odds):
        return np.nan
    o = float(american_odds)
    if won == 1:
        return (o / 100.0) if o > 0 else (100.0 / abs(o))
    return -1.0

favorites["profit_1u"] = favorites.apply(
    lambda r: profit_per_1_unit_bet(r["outcome_price"], int(r["favorite_won"])),
    axis=1
)

def wilson_interval(k, n, z=1.96):
    """Wilson score interval for a binomial proportion."""
    if n == 0:
        return (np.nan, np.nan)
    phat = k / n
    denom = 1 + (z**2)/n
    center = (phat + (z**2)/(2*n)) / denom
    half = (z * np.sqrt((phat*(1-phat) + (z**2)/(4*n)) / n)) / denom
    return (max(0, center-half), min(1, center+half))

def make_figs(fav_df):
    bins = np.linspace(0, 1, 21)
    tmp = fav_df.dropna(subset=["implied_prob"]).copy()
    tmp["prob_bin"] = pd.cut(tmp["implied_prob"], bins=bins, include_lowest=True)

    calib = (
        tmp.groupby("prob_bin", as_index=False)
        .agg(
            implied_prob_mean=("implied_prob", "mean"),
            wins=("favorite_won", "sum"),
            n=("favorite_won", "size"),
            actual_win_rate=("favorite_won", "mean"),
        )
    )
    calib = calib[calib["n"] >= 10].copy()  # reduce noise

    ci = calib.apply(lambda r: wilson_interval(int(r["wins"]), int(r["n"])), axis=1)
    calib["ci_low"] = [c[0] for c in ci]
    calib["ci_high"] = [c[1] for c in ci]
    calib["err_low"] = calib["actual_win_rate"] - calib["ci_low"]
    calib["err_high"] = calib["ci_high"] - calib["actual_win_rate"]

    fig_calib = go.Figure()

    fig_calib.add_trace(go.Scatter(
        x=calib["implied_prob_mean"],
        y=calib["actual_win_rate"],
        mode="markers",
        marker=dict(size=np.clip(calib["n"], 10, 120), opacity=0.85),
        error_y=dict(
            type="data",
            symmetric=False,
            array=calib["err_high"],
            arrayminus=calib["err_low"]
        ),
        customdata=np.stack([calib["n"], calib["wins"], calib["ci_low"], calib["ci_high"]], axis=1),
        hovertemplate=(
            "<b>Bin</b><br>"
            "Mean implied: %{x:.3f}<br>"
            "Actual win rate: %{y:.3f}<br>"
            "95% CI: [%{customdata[2]:.3f}, %{customdata[3]:.3f}]<br>"
            "n=%{customdata[0]:.0f}, wins=%{customdata[1]:.0f}"
            "<extra></extra>"
        ),
        name="Binned calibration",
    ))

    fig_calib.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

    fig_calib.update_layout(
        title="Calibration: Implied Probability vs Actual Win Rate (with 95% CI)",
        xaxis_title="Mean implied probability (favorite)",
        yaxis_title="Actual win rate (favorite)",
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        paper_bgcolor="white",
        plot_bgcolor="white",
        margin=dict(l=50, r=30, t=60, b=50),
    )

    tmp2 = fav_df.dropna(subset=["implied_prob"]).copy()
    tmp2 = tmp2[tmp2["implied_prob"] >= 0.5].copy()
    tmp2["prob_bin"] = pd.cut(tmp2["implied_prob"], bins=np.linspace(0.5, 1.0, 11), include_lowest=True)

    bias = (
        tmp2.groupby("prob_bin", as_index=False)
        .agg(
            mean_implied=("implied_prob", "mean"),
            actual_win_rate=("favorite_won", "mean"),
            n_games=("favorite_won", "size"),
        )
    )
    bias["bias"] = bias["actual_win_rate"] - bias["mean_implied"]

    fig_bias = go.Figure()
    fig_bias.add_trace(go.Bar(
        x=bias["mean_implied"],
        y=bias["bias"],
        customdata=np.stack([bias["n_games"], bias["actual_win_rate"], bias["mean_implied"]], axis=1),
        hovertemplate=(
            "<b>Favorite strength bin</b><br>"
            "Mean implied: %{customdata[2]:.3f}<br>"
            "Actual win rate: %{customdata[1]:.3f}<br>"
            "<b>Bias (actual - implied): %{y:.3f}</b><br>"
            "Games: %{customdata[0]:.0f}"
            "<extra></extra>"
        ),
        name="Bias"
    ))

    fig_bias.add_hline(y=0)
    fig_bias.update_layout(
        title="Bias by Favorite Strength (Actual − Implied)",
        xaxis_title="Mean implied probability (favorite)",
        yaxis_title="Bias (Actual − Implied)",
        paper_bgcolor="white",
        plot_bgcolor="white",
        margin=dict(l=50, r=30, t=60, b=50),
    )

    fig_dist = px.histogram(
        fav_df.dropna(subset=["implied_prob"]),
        x="implied_prob",
        color="favorite_won",
        nbins=20,
        title="Distribution of Implied Probabilities by Outcome",
        labels={"implied_prob": "Implied probability", "favorite_won": "Favorite won"},
    )
    fig_dist.update_layout(
        paper_bgcolor="white",
        plot_bgcolor="white",
        margin=dict(l=50, r=30, t=60, b=50),
    )

    tmp3 = fav_df.dropna(subset=["implied_prob", "profit_1u"]).copy()
    tmp3 = tmp3[tmp3["implied_prob"] >= 0.5].copy()
    tmp3["prob_bin"] = pd.cut(tmp3["implied_prob"], bins=np.linspace(0.5, 1.0, 11), include_lowest=True)

    roi = (
        tmp3.groupby("prob_bin", as_index=False)
        .agg(
            mean_implied=("implied_prob", "mean"),
            avg_profit=("profit_1u", "mean"),
            n=("profit_1u", "size"),
        )
    )

    fig_roi = go.Figure()
    fig_roi.add_trace(go.Bar(
        x=roi["mean_implied"],
        y=roi["avg_profit"],
        customdata=np.stack([roi["n"]], axis=1),
        hovertemplate=(
            "Mean implied: %{x:.3f}<br>"
            "Avg profit per bet: %{y:.3f}<br>"
            "n=%{customdata[0]:.0f}"
            "<extra></extra>"
        ),
        name="ROI"
    ))
    fig_roi.add_hline(y=0)
    fig_roi.update_layout(
        title="ROI by Favorite Strength (1 unit bets on favorites)",
        xaxis_title="Mean implied probability (favorite)",
        yaxis_title="Avg profit per bet",
        paper_bgcolor="white",
        plot_bgcolor="white",
        margin=dict(l=50, r=30, t=60, b=50),
    )

    return fig_calib, fig_bias, fig_dist, fig_roi

def make_conclusion(fav_df):
    n = len(fav_df)
    if n == 0:
        return "No data for the selected filters."

    win_rate = fav_df["favorite_won"].mean()
    mean_p = fav_df["implied_prob"].mean()
    roi = fav_df["profit_1u"].mean()

    brier = np.mean((fav_df["implied_prob"] - fav_df["favorite_won"])**2)

    tmp = fav_df.dropna(subset=["implied_prob"]).copy()
    tmp = tmp[tmp["implied_prob"] >= 0.5].copy()
    tmp["prob_bin"] = pd.cut(tmp["implied_prob"], bins=np.linspace(0.5, 1.0, 11), include_lowest=True)
    bias = (
        tmp.groupby("prob_bin")
        .agg(mean_implied=("implied_prob", "mean"),
             actual=("favorite_won", "mean"),
             n=("favorite_won", "size"))
        .reset_index()
    )
    bias["bias"] = bias["actual"] - bias["mean_implied"]
    bias = bias[bias["n"] >= 10].copy()

    if len(bias) > 0:
        worst = bias.iloc[(bias["bias"].abs()).argmax()]
        worst_txt = (
            f"Biggest deviation shows up around **implied ≈ {worst['mean_implied']:.2f}** "
            f"(bias **{worst['bias']:+.3f}**, n={int(worst['n'])})."
        )
    else:
        worst_txt = "Not enough games per bin to reliably identify the biggest bias bucket (try broader filters)."

    return f"""
### Auto summary (updates with filters)

- You analyzed **{n:,}** favorite bets.
- Favorites won **{win_rate:.1%}** of the time vs an average implied probability of **{mean_p:.1%}**.
- **Calibration** looks {'close' if abs(win_rate-mean_p) < 0.02 else 'off'} overall (difference **{(win_rate-mean_p):+.2%}**).
- **Brier score** (lower is better): **{brier:.3f}**.
- **ROI** from betting 1 unit on every favorite: **{roi:+.3f}** units per bet.

{worst_txt}
"""

app = Dash(__name__)

all_seasons = sorted([int(x) for x in favorites["season"].dropna().unique()])
all_books = sorted([str(x) for x in favorites["bookmaker_title"].dropna().unique()])

app.layout = html.Div(
    style={
        "minHeight": "100vh",
        "backgroundColor": "#f6f7fb",
        "padding": "28px 16px",
        "fontFamily": '-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif',
        "color": "#111827",
    },
    children=[
        html.Div(
            style={"maxWidth": "1100px", "margin": "0 auto"},
            children=[

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "22px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[
                        html.H1("NBA Betting Odds vs Outcomes",
                                style={"margin": "0 0 6px 0", "fontWeight": "900", "letterSpacing": "-0.5px"}),
                        html.Div(
                            "Filters update all plots + the written conclusion below.",
                            style={"color": "#6b7280"}
                        ),

                        html.Div(
                            style={
                                "display": "grid",
                                "gridTemplateColumns": "1fr 2fr",
                                "gap": "12px",
                                "marginTop": "14px",
                            },
                            children=[
                                html.Div([
                                    html.Div("Season", style={"fontWeight": "700", "fontSize": "13px", "color": "#374151", "marginBottom": "6px"}),
                                    dcc.Dropdown(
                                        id="season_dd",
                                        options=[{"label": "All seasons", "value": "ALL"}] + [{"label": str(s), "value": s} for s in all_seasons],
                                        value="ALL",
                                        clearable=False,
                                    )
                                ]),
                                html.Div([
                                    html.Div("Bookmaker", style={"fontWeight": "700", "fontSize": "13px", "color": "#374151", "marginBottom": "6px"}),
                                    dcc.Dropdown(
                                        id="book_dd",
                                        options=[{"label": "All bookmakers", "value": "ALL"}] + [{"label": b, "value": b} for b in all_books],
                                        value="ALL",
                                        clearable=False,
                                    )
                                ]),
                            ],
                        ),
                    ],
                ),

                html.Div(id="kpi_row", style={
                    "display": "grid",
                    "gridTemplateColumns": "repeat(4, minmax(0, 1fr))",
                    "gap": "12px",
                    "marginBottom": "14px",
                }),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "18px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[
                        dcc.Markdown(id="conclusion_md")
                    ],
                ),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "16px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[dcc.Graph(id="fig_calib")]
                ),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "16px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[dcc.Graph(id="fig_bias")]
                ),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "16px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[dcc.Graph(id="fig_dist")]
                ),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "16px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[dcc.Graph(id="fig_roi")]
                ),

                html.Div(
                    style={
                        "backgroundColor": "white",
                        "borderRadius": "16px",
                        "padding": "20px",
                        "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
                        "border": "1px solid rgba(0,0,0,0.06)",
                        "marginBottom": "14px",
                    },
                    children=[
                        html.H3(
                            "Glossary",
                            style={"marginTop": "0", "fontWeight": "900", "color": "#111827"}
                        ),

                        dcc.Markdown(
                            """
                **Implied Probability**  
                The probability of an outcome implied by betting odds.  
                For example, -150 odds imply about a **60% chance** of winning.

                **Favorite**  
                The team with the **higher implied probability** (lower payout, expected to win).

                **Calibration**  
                How closely implied probabilities match real outcomes.  
                Perfect calibration means a 60% favorite wins about **60% of the time**.

                **Bias (Actual − Implied)**  
                The difference between how often favorites actually win and how often sportsbooks imply they should win.  
                - Positive → favorites win **more** than expected  
                - Negative → favorites are **overvalued**

                **ROI (1 Unit)**  
                Average profit or loss from betting **1 unit on every favorite**.  
                - Positive ROI → profitable strategy  
                - Negative ROI → losing strategy

                **Brier Score**  
                A measure of prediction accuracy (lower is better).  
                It penalizes confident but incorrect predictions more heavily.

                **Wilson Confidence Interval**  
                A statistically robust range showing uncertainty in win-rate estimates, especially helpful when sample sizes are small.
                            """,
                            style={"fontSize": "14px", "color": "#374151"},
                        ),
                    ],
                ),

                html.Div(
                    "Odds: The Odds API • Results: balldontlie • Built with Dash + SQLite",
                    style={"textAlign": "center", "color": "#6b7280", "fontSize": "13px", "padding": "6px 0"},
                ),
            ],
        )
    ]
)

def kpi_card(title, value, subtitle):
    return html.Div(
        style={
            "backgroundColor": "white",
            "borderRadius": "16px",
            "padding": "14px 16px",
            "boxShadow": "0 6px 18px rgba(0,0,0,0.06)",
            "border": "1px solid rgba(0,0,0,0.06)",
        },
        children=[
            html.Div(title, style={"fontSize": "13px", "fontWeight": "700", "color": "#374151"}),
            html.Div(value, style={"fontSize": "28px", "fontWeight": "900", "color": "#111827", "marginTop": "6px"}),
            html.Div(subtitle, style={"fontSize": "13px", "color": "#6b7280", "marginTop": "2px"}),
        ],
    )

@app.callback(
    Output("fig_calib", "figure"),
    Output("fig_bias", "figure"),
    Output("fig_dist", "figure"),
    Output("fig_roi", "figure"),
    Output("conclusion_md", "children"),
    Output("kpi_row", "children"),
    Input("season_dd", "value"),
    Input("book_dd", "value"),
)
def update(season_val, book_val):
    f = favorites.copy()

    if season_val != "ALL":
        f = f[f["season"] == int(season_val)]
    if book_val != "ALL":
        f = f[f["bookmaker_title"] == str(book_val)]

    fig_calib, fig_bias, fig_dist, fig_roi = make_figs(f)
    conclusion = make_conclusion(f)

    n = len(f)
    kpis = [
        kpi_card("Bets", f"{n:,}", "favorite bets analyzed"),
        kpi_card("Bookmakers", f"{f.bookmaker_title.nunique()}", "books included"),
        kpi_card("Favorite Win %", f"{f.favorite_won.mean():.1%}" if n else "—", "actual win rate"),
        kpi_card("ROI (1u)", f"{f.profit_1u.mean():+.3f}" if n else "—", "avg profit per bet"),
    ]

    return fig_calib, fig_bias, fig_dist, fig_roi, conclusion, kpis



C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\929345476.py:42: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\929345476.py:106: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\929345476.py:166: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\Hieut\AppData\Local\Temp\ipykernel_24596\929345476.py:217: FutureWarning:
